In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Amazonas - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_AM.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Amazonas - IDH,Amazonas - value,Amazonas - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - Produção de Cimento (t),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - Consumo de Cimento (t)
0,2003-1,0.695898,0.330279,8.630942,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,51.313080,4.754469e+07,2.519796e+06,12.490621,4.318518e+07,28.193
1,2003-2,0.696043,0.331137,8.624872,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,51.554195,4.760190e+07,2.521321e+06,12.492164,4.320154e+07,31.226
2,2003-3,0.696187,0.331306,8.618803,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,51.523306,4.765911e+07,2.522846e+06,12.493707,4.321789e+07,33.484
3,2003-4,0.696332,0.331791,8.612734,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,51.639628,4.771632e+07,2.524370e+06,12.495250,4.323425e+07,31.596
4,2003-5,0.696476,0.332189,8.606665,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,51.838456,4.777353e+07,2.525895e+06,12.496793,4.325060e+07,32.919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,0.746275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.352061,NaN,NaN,NaN,NaN,66.077
236,2022-9,NaN,0.746582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.160630,NaN,NaN,NaN,NaN,58.590
237,2022-10,NaN,0.745406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.937187,NaN,NaN,NaN,NaN,58.793
238,2022-11,NaN,0.742703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.889598,NaN,NaN,NaN,NaN,53.509


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Amazonas - IDH,Amazonas - value,Amazonas - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - Produção de Cimento (t),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado
0,-2.017357,-1.841109,-0.841381,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.246822,-1.725325,0.115495,1.032096,-1.955531
1,-1.988164,-1.831498,-0.844328,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.217639,-1.704856,0.145069,1.057757,-1.907364
2,-1.958970,-1.829607,-0.847276,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.221378,-1.684387,0.174644,1.083417,-1.859197
3,-1.929776,-1.824180,-0.850223,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.207299,-1.663918,0.204218,1.109077,-1.811030
4,-1.900583,-1.819720,-0.853171,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.183234,-1.643449,0.233793,1.134738,-1.762863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.146285,0.624622,1.092028,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-2.063199,1.193835,-1.214518,-0.726615,1.324047
188,1.136919,0.664868,1.088543,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-2.019550,1.186279,-1.186673,-0.718777,1.320189
189,1.127552,0.713988,1.085058,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.981485,1.178724,-1.158827,-0.710938,1.316331
190,1.118186,0.765366,1.081573,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.940641,1.171169,-1.130982,-0.703100,1.312472


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      34.293
1      48.623
2      34.555
3      57.805
4      39.036
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Amazonas - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Amazonas - IDH,Amazonas - value,Amazonas - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - Produção de Cimento (t),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado
0,-2.017357,-1.841109,-0.841381,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.246822,-1.725325,0.115495,1.032096,-1.955531
1,-1.988164,-1.831498,-0.844328,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.217639,-1.704856,0.145069,1.057757,-1.907364
2,-1.958970,-1.829607,-0.847276,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.221378,-1.684387,0.174644,1.083417,-1.859197
3,-1.929776,-1.824180,-0.850223,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.207299,-1.663918,0.204218,1.109077,-1.811030
4,-1.900583,-1.819720,-0.853171,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.183234,-1.643449,0.233793,1.134738,-1.762863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.448925,0.452650,1.390527,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-0.823205,1.169567,-1.503954,-1.462487,0.955680
158,1.452046,0.404414,1.372620,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-0.905554,1.175278,-1.511151,-1.435811,0.975593
159,1.455166,0.365711,1.354712,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-0.983877,1.180989,-1.518349,-1.409135,0.995507
160,1.458287,0.316318,1.336804,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-1.057189,1.186700,-1.525547,-1.382459,1.015420


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      34.293
1      48.623
2      34.555
3      57.805
4      39.036
        ...  
157    44.068
158    62.847
159    30.374
160    60.003
161    81.014
Name: Amazonas - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Amazonas - IDH,Amazonas - value,Amazonas - Desemprego,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Amazonas - Produção de Cimento (t),Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado
126,0.555057,1.274691,0.945132,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.563099,0.801569,-0.466491,-0.864199,0.860635
127,0.563987,1.302840,0.969467,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.502599,0.817293,-0.522440,-0.912686,0.854184
128,0.572918,1.321485,0.993802,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.402261,0.833017,-0.578389,-0.961174,0.847732
129,0.581848,1.339108,1.018136,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.355033,0.848741,-0.634338,-1.009662,0.841281
130,0.590779,1.353338,1.042471,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.291284,0.864466,-0.690287,-1.058150,0.834830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.146285,0.624622,1.092028,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-2.063199,1.193835,-1.214518,-0.726615,1.324047
188,1.136919,0.664868,1.088543,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-2.019550,1.186279,-1.186673,-0.718777,1.320189
189,1.127552,0.713988,1.085058,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.981485,1.178724,-1.158827,-0.710938,1.316331
190,1.118186,0.765366,1.081573,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.940641,1.171169,-1.130982,-0.703100,1.312472


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
#     train, train_val = validation_splitter(train_input, 7)
#     target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
#                         validation_data=(train_val,
#                                          target_val),
                        validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2617676647, 1786315749, 1625686316, 122779991, 1317391866, 2111767164, 3590399031, 1852957515, 155329372, 2349571259, 2050313068, 2318193615, 1569886954, 3217919937, 2686486965, 368206856, 1965403989, 2804069892, 2281942797, 2428904084, 2744249318, 2955223404, 1768486850, 3312096685, 606325584, 643726119, 2793864362, 1110418197, 1135789268, 1044883584, 3703318474, 2139561888, 73773203, 1016063984, 3975204820, 380145783, 1994270290, 1357424051, 2547396210, 3045176606, 1788280379, 3855320443, 379419894, 2769115701, 1747937358, 1883070224, 2313822975, 556417946, 3619278844, 3310024491]


Step: 0 ___________________________________________
val_loss: 1028.809326171875
winner_seed: 2617676647


Step: 1 ___________________________________________
val_loss: 131.56533813476562
winner_seed: 1786315749


Step: 2 ___________________________________________
val_loss: 250.30612182617188


Step: 3 ___________________________________________
val_loss: 131.5365447998047
winner_seed: 122779991


Step: 

In [35]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 207ms/step - loss: 3801.4722 - val_loss: 463.2016
Epoch 2/10000
4/4 [==============================] - 0s 11ms/step - loss: 892.3818 - val_loss: 411.4696
Epoch 3/10000
4/4 [==============================] - 0s 10ms/step - loss: 189.5846 - val_loss: 488.3984
Epoch 4/10000
4/4 [==============================] - 0s 10ms/step - loss: 225.5480 - val_loss: 724.4560
Epoch 5/10000
4/4 [==============================] - 0s 10ms/step - loss: 152.1875 - val_loss: 274.2308
Epoch 6/10000
4/4 [==============================] - 0s 11ms/step - loss: 123.9729 - val_loss: 633.5391
Epoch 7/10000
4/4 [==============================] - 0s 10ms/step - loss: 152.0112 - val_loss: 171.4825
Epoch 8/10000
4/4 [==============================] - 0s 10ms/step - loss: 115.8823 - val_loss: 159.1029
Epoch 9/10000
4/4 [==============================] - 0s 11ms/step - loss: 107.7282 - val_loss: 155.7559
Epoch 10/10000
4/4 [==============================] - 0s 10ms/

4/4 [==============================] - 0s 16ms/step - loss: 105.5572 - val_loss: 150.8546
Epoch 80/10000
4/4 [==============================] - 0s 16ms/step - loss: 89.5204 - val_loss: 148.2397
Epoch 81/10000
4/4 [==============================] - 0s 16ms/step - loss: 95.9407 - val_loss: 174.5942
Epoch 82/10000
4/4 [==============================] - 0s 15ms/step - loss: 89.6653 - val_loss: 162.6858
Epoch 83/10000
4/4 [==============================] - 0s 12ms/step - loss: 87.5718 - val_loss: 201.4940
Epoch 84/10000
4/4 [==============================] - 0s 14ms/step - loss: 94.6800 - val_loss: 184.5683
Epoch 85/10000
4/4 [==============================] - 0s 13ms/step - loss: 95.4208 - val_loss: 163.1422
Epoch 86/10000
4/4 [==============================] - 0s 13ms/step - loss: 93.7273 - val_loss: 154.4419
Epoch 87/10000
4/4 [==============================] - 0s 15ms/step - loss: 110.7551 - val_loss: 192.2071
Epoch 88/10000
4/4 [==============================] - 0s 15ms/step - loss: 88

4/4 [==============================] - 0s 16ms/step - loss: 90.6215 - val_loss: 150.4695
Epoch 158/10000
4/4 [==============================] - 0s 14ms/step - loss: 96.8775 - val_loss: 149.4203
Epoch 159/10000
4/4 [==============================] - 0s 15ms/step - loss: 100.4720 - val_loss: 171.4743
Epoch 160/10000
4/4 [==============================] - 0s 19ms/step - loss: 99.1431 - val_loss: 155.2779
Epoch 161/10000
4/4 [==============================] - 0s 15ms/step - loss: 91.4101 - val_loss: 222.7766
Epoch 162/10000
4/4 [==============================] - 0s 17ms/step - loss: 103.4104 - val_loss: 171.5561
Epoch 163/10000
4/4 [==============================] - 0s 16ms/step - loss: 89.9743 - val_loss: 171.5096
Epoch 164/10000
4/4 [==============================] - 0s 15ms/step - loss: 88.6674 - val_loss: 159.8130
Epoch 165/10000
4/4 [==============================] - 0s 16ms/step - loss: 90.2827 - val_loss: 154.6325
Epoch 166/10000
4/4 [==============================] - 0s 14ms/step -

4/4 [==============================] - 0s 12ms/step - loss: 99.6265 - val_loss: 158.3578
Epoch 236/10000
4/4 [==============================] - 0s 14ms/step - loss: 86.2274 - val_loss: 178.8079
Epoch 237/10000
4/4 [==============================] - 0s 15ms/step - loss: 85.5615 - val_loss: 154.0782
Epoch 238/10000
4/4 [==============================] - 0s 15ms/step - loss: 93.2113 - val_loss: 142.6126
Epoch 239/10000
4/4 [==============================] - 0s 16ms/step - loss: 87.8984 - val_loss: 157.5003
Epoch 240/10000
4/4 [==============================] - 0s 15ms/step - loss: 86.3883 - val_loss: 142.3350
Epoch 241/10000
4/4 [==============================] - 0s 13ms/step - loss: 86.2100 - val_loss: 174.3027
Epoch 242/10000
4/4 [==============================] - 0s 13ms/step - loss: 89.9433 - val_loss: 155.2986
Epoch 243/10000
4/4 [==============================] - 0s 14ms/step - loss: 81.7236 - val_loss: 183.3485
Epoch 244/10000
4/4 [==============================] - 0s 14ms/step - l

4/4 [==============================] - 0s 16ms/step - loss: 85.6992 - val_loss: 151.6674
Epoch 314/10000
4/4 [==============================] - 0s 17ms/step - loss: 81.8250 - val_loss: 149.4155
Epoch 315/10000
4/4 [==============================] - 0s 16ms/step - loss: 83.6893 - val_loss: 166.8391
Epoch 316/10000
4/4 [==============================] - 0s 15ms/step - loss: 99.1011 - val_loss: 157.2354
Epoch 317/10000
4/4 [==============================] - 0s 12ms/step - loss: 82.5857 - val_loss: 151.3750
Epoch 318/10000
4/4 [==============================] - 0s 13ms/step - loss: 83.0236 - val_loss: 160.7309
Epoch 319/10000
4/4 [==============================] - 0s 14ms/step - loss: 83.0258 - val_loss: 140.1928
Epoch 320/10000
4/4 [==============================] - 0s 15ms/step - loss: 88.2355 - val_loss: 143.0492
Epoch 321/10000
4/4 [==============================] - 0s 15ms/step - loss: 89.8788 - val_loss: 150.1123
Epoch 322/10000
4/4 [==============================] - 0s 14ms/step - l

...

4/4 [==============================] - 0s 16ms/step - loss: 17.8792 - val_loss: 123.8827
Epoch 1170/10000
4/4 [==============================] - 0s 16ms/step - loss: 16.4379 - val_loss: 125.0174
Epoch 1171/10000
4/4 [==============================] - 0s 15ms/step - loss: 18.1416 - val_loss: 125.1175
Epoch 1172/10000
4/4 [==============================] - 0s 13ms/step - loss: 27.3347 - val_loss: 124.6152
Epoch 1173/10000
4/4 [==============================] - 0s 13ms/step - loss: 19.9299 - val_loss: 125.4418
Epoch 1174/10000
4/4 [==============================] - 0s 14ms/step - loss: 21.7677 - val_loss: 125.3264
Epoch 1175/10000
4/4 [==============================] - 0s 13ms/step - loss: 18.5103 - val_loss: 125.0418
Epoch 1176/10000
4/4 [==============================] - 0s 14ms/step - loss: 23.6645 - val_loss: 126.2771
Epoch 1177/10000
4/4 [==============================] - 0s 16ms/step - loss: 25.7070 - val_loss: 128.6848
Epoch 1178/10000
4/4 [==============================] - 0s 15ms

4/4 [==============================] - 0s 16ms/step - loss: 17.6710 - val_loss: 126.6208
Epoch 1247/10000
4/4 [==============================] - 0s 15ms/step - loss: 15.1464 - val_loss: 125.9646
Epoch 1248/10000
4/4 [==============================] - 0s 15ms/step - loss: 19.2384 - val_loss: 126.7104
Epoch 1249/10000
4/4 [==============================] - 0s 15ms/step - loss: 21.2036 - val_loss: 124.1647
Epoch 1250/10000
4/4 [==============================] - 0s 13ms/step - loss: 13.0720 - val_loss: 124.3452
Epoch 1251/10000
4/4 [==============================] - 0s 13ms/step - loss: 16.7391 - val_loss: 125.0050
Epoch 1252/10000
4/4 [==============================] - 0s 14ms/step - loss: 12.1380 - val_loss: 125.3779
Epoch 1253/10000
4/4 [==============================] - 0s 13ms/step - loss: 15.8573 - val_loss: 124.5470
Epoch 1254/10000
4/4 [==============================] - 0s 19ms/step - loss: 14.7522 - val_loss: 125.1561
Epoch 1255/10000
4/4 [==============================] - 0s 19ms

4/4 [==============================] - 0s 13ms/step - loss: 10.3460 - val_loss: 125.2581
Epoch 1324/10000
4/4 [==============================] - 0s 13ms/step - loss: 10.3247 - val_loss: 124.7442
Epoch 1325/10000
4/4 [==============================] - 0s 15ms/step - loss: 10.5920 - val_loss: 125.7564
Epoch 1326/10000
4/4 [==============================] - 0s 16ms/step - loss: 11.7222 - val_loss: 125.7858
Epoch 1327/10000
4/4 [==============================] - 0s 14ms/step - loss: 9.9062 - val_loss: 124.8533
Epoch 1328/10000
4/4 [==============================] - 0s 16ms/step - loss: 11.8844 - val_loss: 125.5838
Epoch 1329/10000
4/4 [==============================] - 0s 17ms/step - loss: 11.7976 - val_loss: 126.7513
Epoch 1330/10000
4/4 [==============================] - 0s 16ms/step - loss: 13.1606 - val_loss: 125.5230
Epoch 1331/10000
4/4 [==============================] - 0s 13ms/step - loss: 12.7857 - val_loss: 125.7974
Epoch 1332/10000
4/4 [==============================] - 0s 12ms/

4/4 [==============================] - 0s 10ms/step - loss: 8.6244 - val_loss: 125.8601
Epoch 1401/10000
4/4 [==============================] - 0s 11ms/step - loss: 9.6605 - val_loss: 125.2790
Epoch 1402/10000
4/4 [==============================] - 0s 11ms/step - loss: 10.3650 - val_loss: 124.4523
Epoch 1403/10000
4/4 [==============================] - 0s 11ms/step - loss: 9.4258 - val_loss: 125.5981
Epoch 1404/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.6671 - val_loss: 125.9559
Epoch 1405/10000
4/4 [==============================] - 0s 10ms/step - loss: 8.8668 - val_loss: 124.9609
Epoch 1406/10000
4/4 [==============================] - 0s 10ms/step - loss: 10.7213 - val_loss: 127.1512
Epoch 1407/10000
4/4 [==============================] - 0s 10ms/step - loss: 9.6600 - val_loss: 125.3569
Epoch 1408/10000
4/4 [==============================] - 0s 10ms/step - loss: 10.6910 - val_loss: 125.8911
Epoch 1409/10000
4/4 [==============================] - 0s 10ms/step

4/4 [==============================] - 0s 13ms/step - loss: 9.6718 - val_loss: 127.4626
Epoch 1479/10000
4/4 [==============================] - 0s 13ms/step - loss: 5.7837 - val_loss: 124.8876
Epoch 1480/10000
4/4 [==============================] - 0s 13ms/step - loss: 7.1919 - val_loss: 125.2848
Epoch 1481/10000
4/4 [==============================] - 0s 14ms/step - loss: 5.3290 - val_loss: 125.0247
Epoch 1482/10000
4/4 [==============================] - 0s 14ms/step - loss: 4.6681 - val_loss: 125.6514
Epoch 1483/10000
4/4 [==============================] - 0s 14ms/step - loss: 6.2754 - val_loss: 125.5466
Epoch 1484/10000
4/4 [==============================] - 0s 15ms/step - loss: 6.7915 - val_loss: 127.2256
Epoch 1485/10000
4/4 [==============================] - 0s 17ms/step - loss: 5.8043 - val_loss: 126.4082
Epoch 1486/10000
4/4 [==============================] - 0s 16ms/step - loss: 7.4875 - val_loss: 125.9010
Epoch 1487/10000
4/4 [==============================] - 0s 16ms/step - l

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 17ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,54.116486,53.764915,53.484383,52.785034,52.095299,51.294693,50.148739,48.498917,48.287144,47.865372,47.078323,45.946938,46.588814,44.501801,44.278412,44.790588,44.453194,43.613319,40.6488,40.16861,39.766251,39.191334,39.83194,39.386425,39.726414,39.069305,39.658333,39.080589,39.256279,38.945053
Target,64.862,49.885,53.13,57.796,47.026,74.919,34.733,15.022,20.879,36.409,22.277,20.529,37.061,23.387,38.399,39.652,40.204,21.625,42.731,36.026,34.609,40.787,39.823,39.957,52.048,53.567,46.239,51.066,51.684,44.907
Error,10.745514,3.879917,0.354382,5.010967,5.069298,23.624306,15.415737,33.476917,27.408144,11.456371,24.801323,25.417938,9.527813,21.114801,5.879414,5.138588,4.249195,21.988319,2.082199,4.142609,5.157249,1.595665,0.008938,0.570576,12.321587,14.497696,6.580666,11.985413,12.427719,5.961948


In [38]:
display(mae)
display(mape)

11.06304

0.38667488

In [42]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [43]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[605.3663]] - Target[497.4669999999999]| =  Error: [[107.89926]]; MAPE:[[0.21689732]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[507.21945]] - Target[434.26099999999997]| =  Error: [[72.958466]]; MAPE:[[0.16800603]]
1/1 [==============================] - 0s 15ms/step
Ano-5: |Prediction[[235.73598]] - Target[299.511]| =  Error: [[63.77501]]; MAPE:[[0.21293046]]


[array([[107.89926]], dtype=float32),
 array([[72.958466]], dtype=float32),
 array([[63.77501]], dtype=float32)]

81.54424

0.19927794